In [ ]:
import mysql.connector
import pandas as pd

In [ ]:
mydb = mysql.connector.connect(
host = 'localhost',
user = 'root',
passwd = '*',
database = 'world')

In [ ]:
kursor = mydb.cursor()
def sql(query) :
    kursor.execute(query)
    hasil = kursor.fetchall()
    data = pd.DataFrame(data = hasil, columns = kursor.column_names)
    return data

Tampilkan 10 nama negara (Name) dan jumlah bahasa (language) baik yang official maupun tidak di tiap negara, dari tabel Country dan CountryLanguage! Tampilkan hanya negara yang jumlah bahasanya di atas rata-rata jumlah bahasa tiap negara! Urutkan dari negara yang jumlah bahasanya paling banyak!

In [ ]:
sql('''
    WITH Country_Language AS (
    SELECT C.Name AS Country_Name,
    COUNT(CL.Language) AS Total_Language
    FROM Country C
    LEFT JOIN Countrylanguage CL
    ON C.Code = CL.CountryCode
    GROUP BY Country_Name)
SELECT * FROM Country_Language
WHERE Total_Language > (SELECT AVG(Total_Language) FROM Country_Language)
ORDER BY Total_Language DESC
LIMIT 10;
''')

,Country_Name,Total_Language
0,United States,12
1,Russian Federation,12
2,India,12
3,China,12
4,Canada,12
5,South Africa,11
6,Tanzania,11
7,Nigeria,10
8,Mozambique,10
9,Iran,10


In [ ]:
sql('''
with 
country_language as(
    select country.name as country_name, count(countrylanguage.language) as total_language
    from country
    join countrylanguage
    on country.code=countrylanguage.countrycode
    group by 1
),
rata_bahasa as(
select avg(total_language) as rata from country_language 
)
select * from country_language
where total_language> (select rata from rata_bahasa)
order by 2 desc
limit 10
''')



,country_name,total_language
0,United States,12
1,Russian Federation,12
2,India,12
3,China,12
4,Canada,12
5,South Africa,11
6,Tanzania,11
7,Nigeria,10
8,Mozambique,10
9,Iran,10


Tampilkan 10 nama negara (Name), bentuk pemerintahan (GovernmentForm), jumlah population semua negara (Population), persentase populasi tiap negara, serta nomor index baris tiap negara dari tabel Country!

In [ ]:
sql('''
SELECT Name,
GovernmentForm,
SUM(Population) OVER () AS World_Population,
Population / SUM(Population) OVER () * 100 AS Percentage,
ROW_NUMBER () OVER () AS Row_Index
FROM Country
ORDER BY Percentage DESC
LIMIT 10;
''')

,Name,GovernmentForm,World_Population,Percentage,Row_Index
0,China,People'sRepublic,6078749450,21.0168,42
1,India,Federal Republic,6078749450,16.6755,100
2,United States,Federal Republic,6078749450,4.5792,224
3,Indonesia,Republic,6078749450,3.4893,99
4,Brazil,Federal Republic,6078749450,2.7985,31
5,Pakistan,Republic,6078749450,2.5743,165
6,Russian Federation,Federal Republic,6078749450,2.4172,182
7,Bangladesh,Republic,6078749450,2.1247,22
8,Japan,Constitutional Monarchy,6078749450,2.0845,110
9,Nigeria,Federal Republic,6078749450,1.8344,156


In [ ]:
sql('''
select name, sum(population) over() as world_pop,
governmentform,
population/ sum(population) over() *100 as percentage,
row_number() over() as row_index
from country
order by 4 desc
limit 10;
''')

,name,world_pop,governmentform,percentage,row_index
0,China,6078749450,People'sRepublic,21.0168,42
1,India,6078749450,Federal Republic,16.6755,100
2,United States,6078749450,Federal Republic,4.5792,224
3,Indonesia,6078749450,Republic,3.4893,99
4,Brazil,6078749450,Federal Republic,2.7985,31
5,Pakistan,6078749450,Republic,2.5743,165
6,Russian Federation,6078749450,Federal Republic,2.4172,182
7,Bangladesh,6078749450,Republic,2.1247,22
8,Japan,6078749450,Constitutional Monarchy,2.0845,110
9,Nigeria,6078749450,Federal Republic,1.8344,156


Tampilkan jumlah kota di tiap kawasan regional (Region)! Tampilkan hanya kawasan di Benua Asia dan Eropa, serta tambahkan nomor sesuai jumlah kawasan regional di tiap benua (Continent)! Gunakan tabel City dan Country! Perhatikan contoh hasilnya seperti di bawah ini!

In [ ]:
sql('''
WITH City_Count AS (
SELECT CountryCode,
COUNT(Name) AS Number_of_City
FROM City
GROUP BY 1)
SELECT C.Continent,
C.Region,
SUM(CT.Number_of_City) AS Number_of_City,
ROW_NUMBER() OVER (PARTITION BY C.Continent) AS Row_Group
FROM Country C, City_Count CT
WHERE C.Code = CT.CountryCode AND C.Continent IN ('Asia', 'Europe')
GROUP BY 1, 2;
''')

,Continent,Region,Number_of_City,Row_Group
0,Asia,Southern and Central Asia,555,1
1,Asia,Middle East,174,2
2,Asia,Southeast Asia,297,3
3,Asia,Eastern Asia,740,4
4,Europe,Southern Europe,156,1
5,Europe,Western Europe,186,2
6,Europe,Eastern Europe,371,3
7,Europe,Nordic Countries,35,4
8,Europe,Baltic Countries,10,5
9,Europe,British Islands,83,6


Segmen konsumen yang potensial biasanya banyak tinggal di daerah ibukota negara. Meskipun, tidak menutup kemungkinan kota di luar ibukota juga mengalami perkembangan secara pesat, baik secara kuantitas pasar maupun daya beli konsumen.  

GNP suatu negara tidak sedikit yang disumbang perusahaan-perusahaan yang beroperasi di daerah ibukota. Terkadang pabriknya di luar ibukota, namun kantor pusatnya akan ditempatkan di ibukota. Sehingga, warga ibukota merupakan pasar potensial, karena banyak dari mereka yang bekerja di perusahaan-perusahaan besar penyumbang GNP nasional. Maka, GNP kurang lebih juga mencerminkan daya beli warga di ibukota suatu negara.  

Untuk menyusun strategi global, sebuah multinational company membutuhkan data peringkat benua berdasarkan jumlah populasi warga ibukota dan peringkat benua berdasarkan rata-rata GNP semua negara di benua tersebut.  

Tulis query SQL dari tabel yang tersedia database World untuk mendapatkan output tabel seperti di bawah ini! Menurut analisis Anda, benua manakah yang perlu diprioritaskan untuk pemasaran produk perusahaan?  

In [ ]:
sql('''
SELECT C.Continent,
SUM(CT.Population) AS Total_Capital_Population,
AVG(C.GNP) AS Average_GNP,
RANK() OVER (ORDER BY SUM(CT.Population) DESC) AS Rank_Population,
RANK() OVER (ORDER BY AVG(C.GNP) DESC) AS Rank_GNP
FROM City CT INNER JOIN Country C
ON C.Capital = CT.Id
GROUP BY 1
ORDER BY 4,5;
''')

,Continent,Total_Capital_Population,Average_GNP,Rank_Population,Rank_GNP
0,Asia,101322793,150105.725490,1,3
1,Europe,51917214,206497.065217,2,2
2,Africa,47714282,10182.017544,3,6
3,South America,28900527,107991.000000,4,4
4,North America,19120216,261854.789189,5,1
5,Oceania,1142110,15547.211111,6,5


Sebuah multinational company lain memiliki pendekatan strategi yang lain. Bukan dengan pendekatan benua prioritas target pasar, melainkan langsung menyasar negara prioritas. Perusahaan ini juga tidak menekankan kuantitas pasar (market size), tapi lebih ke daya beli pasar (consumer's buying power).  
Salah satu aspek yang mencerminkan daya beli pasar di suatu negara adalah GNP. Asumsinya semakin tinggi GNP suatu negara, maka semakin prioritas menjadi target product marketing. Dengan hanya menyasar negara-negara yang menyumbang 80 persen dari total GNP Global, maka operasi pemasaran akan lebih efisien.  
Perusahaan menugaskan Anda untuk mendapatkan output table seperti di bawah ini dari World Database. Tulis SQL query untuk menampilkan:  
* Nama negara prioritas,
* Persentase GNP tiap negara dibandingkan total GNP Global,
* Nilai kumulatif (Moving Sum) dari persentase GNP hanya sampai 80 persen saja,
* Peringkat berdasarkan persentase GNP, dan
* Kelompokkan negara-negara prioritas ini ke dalam 4 kelompok Market Priority!
* Tampilkan hanya negara-negara yang GNP-nya di atas satu persen!

In [ ]:
sql('''
WITH GNP_Table AS (
SELECT Name AS Country_Name,
ROUND(GNP / SUM(GNP) OVER () * 100, 2) AS GNP_Percentage
FROM Country
ORDER BY GNP_Percentage DESC)
SELECT Country_Name,
GNP_Percentage,
ROUND(SUM(GNP_Percentage) OVER (ORDER BY GNP_Percentage DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW), 2) AS Cumulative_GNP,
RANK() OVER (ORDER BY GNP_Percentage DESC) AS GNP_Rank,
NTILE(4) OVER () AS Market_Priority_1234 FROM GNP_Table
WHERE GNP_Percentage > 1;
''')

,Country_Name,GNP_Percentage,Cumulative_GNP,GNP_Rank,Market_Priority_1234
0,United States,28.99,28.99,1,1
1,Japan,12.90,41.89,2,1
2,Germany,7.27,49.16,3,1
3,France,4.85,54.01,4,1
4,United Kingdom,4.70,58.71,5,2
5,Italy,3.96,62.67,6,2
6,China,3.35,66.02,7,2
7,Brazil,2.65,68.67,8,2
8,Canada,2.04,70.71,9,3
9,Spain,1.88,72.59,10,3
